In [1]:
from data import Data

In [2]:
from yaml import safe_load

origin_station = "NAT"
destination_stations = ["LYO", "CHL", "AUB", "ETO", "DEF"]
date = "04/02/2020"
write_output = False

with open(f"/home/justine/Nextcloud/Cired/Recherche/Econometrie/fail_to_board_probability/f2b_code/f2b/parameters/parameters_{origin_station}.yml") as file:
    parameters = safe_load(file)

data = Data(date, origin_station, destination_stations)


Get feasible runs...


100%|██████████| 11715/11715 [00:01<00:00, 7716.17it/s]


number of too long trips removed: 269


100%|██████████| 11436/11436 [00:00<00:00, 2400463.44it/s]


Get runs info...


100%|██████████| 306/306 [00:00<00:00, 312.11it/s]


Order feasible run lists...


100%|██████████| 11436/11436 [00:00<00:00, 1050597.09it/s]


Construct feasible run pairs...


100%|██████████| 11436/11436 [00:00<00:00, 232829.13it/s]


In [3]:
for run_position in range(len(data.runs)-1):
    run_code = data.runs[run_position]
    next_run_code = data.runs[run_position+1]
    if not set(data.concerned_trips_by_run[run_code]).intersection(data.concerned_trips_by_run[next_run_code]):
        print(f"Chain break between runs at positions {run_position} and {run_position+1}.")


Chain break between runs at positions 302 and 303.
Chain break between runs at positions 304 and 305.


In [4]:
print(data.concerned_trips_by_run[data.runs[302]])
print(data.concerned_trips_by_run[data.runs[303]])

[834557, 677181, 770516, 784585, 466491, 590422, 608230, 611542, 615840, 648312, 727664]
[581681, 622734, 737933, 743182, 831588]


### Hessian verification at starting point.

In [9]:
from numpy import array
f2b = array([0. for _ in range(len(data.runs))])


In [12]:
from likelihood_recursive_blocks import (
    compute_access_individual_likelihoods,
    compute_egress_individual_likelihoods,
    differential_log_likelihood_global_by_component,
    indiv_likelihood_and_auxiliary_egress_update_by_run,
    log_likelihood_global_with_egress_auxiliary_variables_updates,
    gradient_log_likelihood_global,
    hessian_log_likelihood_global,
)

In [59]:
access_individual_likelihoods = compute_access_individual_likelihoods(
        data, parameters
    )
egress_individual_likelihoods = compute_egress_individual_likelihoods(
    data, parameters
)
individual_likelihoods = {}
individual_log_likelihood_gradients = {}
access_auxiliary_variables = {}
egress_auxiliary_variables = {}

log_likelihood_global_with_egress_auxiliary_variables_updates(
    f2b,
    access_individual_likelihoods,
    egress_individual_likelihoods,
    egress_auxiliary_variables,
    individual_likelihoods,
    data,
)
individual_log_likelihood_gradients = {}

gradient = gradient_log_likelihood_global(
    f2b,
    access_individual_likelihoods,
    egress_individual_likelihoods,
    access_auxiliary_variables,
    egress_auxiliary_variables,
    individual_likelihoods,
    individual_log_likelihood_gradients,
    data,
)


(Offline) Compute access individual likelihood...


100%|██████████| 11436/11436 [00:04<00:00, 2335.74it/s]


(Offline) Compute egress individual likelihood...


100%|██████████| 11436/11436 [00:03<00:00, 3355.61it/s]


In [60]:
run_position = 50
run_code = data.runs[run_position]
grad = 0
for trip_id in data.concerned_trips_by_run[run_code]:
    grad += individual_log_likelihood_gradients[trip_id, run_code]
print(grad)
print(gradient[50])

839.8883081240866
839.8883081240866


In [61]:
access_auxiliary_comparison = {}
egress_aux_comparison = {}

differential_log_likelihood_global_by_component(
    run_position,
    f2b,
    access_individual_likelihoods,
    egress_individual_likelihoods,
    access_auxiliary_comparison,
    egress_aux_comparison,
    individual_likelihoods,
    data,
)

839.8883081240866

In [71]:
trip_of_interest = 0
hessian = hessian_log_likelihood_global(        
    f2b,
    egress_individual_likelihoods,
    access_auxiliary_variables,
    egress_auxiliary_variables,
    individual_likelihoods,
    individual_log_likelihood_gradients,
    trip_of_interest,
    data,)

In [72]:
print(hessian[50,50])

-0.9951558728137816


In [73]:
hessian_diag = 0
for trip_id in data.concerned_trips_by_run[run_code]:
    hessian_diag += individual_log_likelihood_gradients[trip_id, run_code]**2
print(hessian_diag)

77545.26217054667


In [69]:
hessian_global_by_hand=array([[0. for _ in range(len(data.runs))] for _ in range(len(data.runs))])
for trip_id in data.trips.index:
    for position, first_run in enumerate(data.feasible_runs_by_trip[trip_id]):
        first_run_index = data.runs.index(first_run)
        hessian_global_by_hand[first_run_index, first_run_index] += (
            -individual_log_likelihood_gradients[trip_id, first_run] ** 2
        )

In [70]:
hessian_global_by_hand[50,50]

-77545.26217054667